## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Yixing Ma, Daniel Lam.   (pair 49)

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
!pip install scikit-learn==1.0

In [2]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted

import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [3]:
df_X, s_y = load_diabetes(return_X_y=True, as_frame=True)

## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [4]:
def get_linear_regression_model( df_X, s_y ):
    x = pd.concat([pd.DataFrame({'intercept': np.ones(len(df_X))}), df_X.reset_index(drop=True)], axis=1)
    beta_hat, residuals, rank, s = np.linalg.lstsq(x, s_y, rcond=-1)
    return beta_hat

In [5]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [6]:
def predict_linear_regression_value( beta_hat, df_X ):
    x = pd.concat([pd.DataFrame({'intercept': np.ones(len(df_X))}), df_X.reset_index(drop=True)], axis=1)
    beta_hat.tolist()
    t_beta_hat = np.array([beta_hat])
    new_beta = t_beta_hat.T
    return np.matmul(x.values, new_beta) 

In [7]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

array([[4.03669521],
       [4.39502589],
       [3.90884703]])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [8]:
def partition_data( df_X, s_y, k ):
    random = np.random.randint(low=1, high=k+1, size=len(df_X))
    df_X_dict = dict()
    s_y_dict = dict()
    
    for k_value in np.unique(random):
        df_X_dict[k_value] = df_X[random == k_value]
        s_y_dict[k_value] = s_y[random == k_value]
        
    return (df_X_dict, s_y_dict)

In [9]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [10]:
# Check fold sizes
total = 0
for i in range(1, 6):
  total += len(dict_k_df_X[i])
  print(i, len(dict_k_df_X[i]))

print(total)
print(len(df_X))


1 91
2 98
3 77
4 75
5 101
442
442


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [11]:
def get_mae( s_y, s_y_hat):
    mae = np.sum(abs(s_y - s_y_hat)) / len(s_y)
    return mae

In [12]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [13]:
mae = np.array([])
for k in dict_k_df_X.keys():
    
    df_X_test = dict_k_df_X[k]
    s_y = dict_k_s_y[k]
    x = []
    y = []
    for i in dict_k_df_X.keys():
        if i != k:
            x.append(dict_k_df_X[i])
            y.append(dict_k_s_y[i])
    x_train, y_train = pd.concat(x), pd.concat(y)

    beta_hat = get_linear_regression_model(x_train, y_train)
    beta_hat.tolist()
    t_beta_hat = np.array([beta_hat])
    new_beta = t_beta_hat.T
    x_train_new = pd.concat([pd.DataFrame({'intercept': np.ones(len(x_train))}), x_train.reset_index(drop=True)], axis=1)
    s_y_hat = df_X_test.apply( lambda x : np.matmul(x_train_new.values, new_beta), axis=1)
    
    mae = np.append( mae, get_mae(s_y,s_y_hat) )  

In [14]:
print("min MAE:{:.2f}, max MAE:{:.2f}, mean MAE:{:.2f}".format(mae.min(),mae.max(),mae.mean()))

min MAE:61.90, max MAE:145.04, mean MAE:77.35


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [15]:
from sklearn import datasets
df_X, s_y = datasets.load_iris(return_X_y=True, as_frame=True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [16]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [17]:
def get_acc( s_1, s_2 ):
    # your code here
    acc = 0
    for x in range(len(s_2)):
        if s_1[x] == s_2[x]:
            acc += 1
    return (acc/len(s_1))
    

In [18]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [19]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
out_low = 0
out_upp = len(df_X) // 5

accuracies = {0.1:[], 0.25:[], 0.3:[], 0.4:[]}
for k in dict_k_df_X.keys():

    # your code here
    train_x = df_X.drop(df_X.index[out_low:out_upp])
    train_y = s_y.drop(s_y.index[out_low:out_upp])
    test_x = df_X[out_low:out_upp]
    test_y = s_y[out_low:out_upp]
    
    out_low += len(df_X) // 5
    out_upp += len(df_X) // 5
    
    in_low = 0
    in_up = len(train_x) // 4
    
    for pos_min_impurity in possible_min_impurity_decrease:
      # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
      clf = tree.DecisionTreeClassifier(criterion="gini", min_impurity_decrease = pos_min_impurity)
      clf.fit(train_x, train_y)
        
      pred_y = clf.predict(test_x)
        
      y_test_list = test_y.values.tolist()
        
      this_acc = get_acc(y_test_list, pred_y)
        
      accuracies[pos_min_impurity].append(this_acc)
        
        
        
    # Use best min impurity decrease to train model
    
    # outer accuracy calculation 
avg_1st = sum(accuracies[0.1])/5
avg_2nd = sum(accuracies[0.25])/5
avg_3rd = sum(accuracies[0.3])/5
avg_4th = sum(accuracies[0.4])/5

avg = {avg_1st:0.1, avg_2nd:0.25, avg_3rd:0.3, avg_4th:0.4}

maximum = max(avg.keys())
hyperparameter = avg[maximum]

print("Testing 0.10 min impurity decrease")
print("    Average accuracy over 4 folds is", avg_1st)
print("Testing 0.25 min impurity decrease")
print("    Average accuracy over 4 folds is", avg_2nd)
print("Testing 0.30 min impurity decrease")
print("    Average accuracy over 4 folds is", avg_3rd)
print("Testing 0.40 min impurity decrease")
print("    Average accuracy over 4 folds is", avg_4th)
print()
print("Best min impurity decrease is", hyperparameter)

for a in avg:
    outer_acc = np.append(outer_acc, a)


    # outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.9266666666666667
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.26
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.06666666666666667
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.0

Best min impurity decrease is 0.1


## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [20]:
print("min:", min(outer_acc))
print("max:", max(outer_acc))
print("Mean:", np.mean(outer_acc))

min: 0.0
max: 0.9266666666666667
Mean: 0.31333333333333335
